In [1]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygris
from census import Census
from us import states
from pptx import Presentation
from pptx.util import Inches, Pt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

class MappingAgent:
    def __init__(self, city_name, state_abbrev, county_name, api_key, output_root):
        """
        Initializes the Agent.
        """
        self.city_name = city_name
        self.state_abbrev = state_abbrev
        self.county_name = county_name
        self.api_key = api_key
        self.c = Census(api_key)
        
        # Setup Output
        self.folder_name = f"{city_name}_{state_abbrev}"
        self.output_path = os.path.join(output_root, self.folder_name)
        if not os.path.exists(self.output_path):
            os.makedirs(self.output_path)
            
        print(f"Agent Initialized for: {city_name}, {state_abbrev}")
        print(f"Output Folder: {self.output_path}")
        
        # Placeholders
        self.city_boundary = None
        self.tracts_geo = None
        
    def _fetch_geometry(self):
        if self.city_boundary is None:
            print("Fetching Boundaries...")
            # City
            ma_places = pygris.places(state=self.state_abbrev, cache=True, year=2021)
            self.city_boundary = ma_places[ma_places['NAME'] == self.city_name]
            
            # County Tracts
            self.tracts_geo = pygris.tracts(state=self.state_abbrev, county=self.county_name, cache=True, year=2021)
            
            # Project to meters for centroid calc
            self.city_boundary = self.city_boundary.to_crs(epsg=3857)
            self.tracts_geo = self.tracts_geo.to_crs(epsg=3857)

    def _get_data_for_city(self, census_vars):
        """
        Fetches Census data, merges with shapes, and filters for City.
        """
        self._fetch_geometry()
        
        # 1. Fetch Data
        state_fips = getattr(states, self.state_abbrev).fips
        raw_data = self.c.acs5.state_county_tract(
            fields=list(census_vars.keys()),
            state_fips=state_fips,
            county_fips=pygris.validate_county(self.state_abbrev, self.county_name),
            tract="*"
        )
        
        # 2. Clean Data
        df = pd.DataFrame(raw_data)
        df = df.rename(columns=census_vars)
        df['GEOID'] = df['state'] + df['county'] + df['tract']
        
        # 3. Merge with Geometry
        county_map = self.tracts_geo.merge(df, on="GEOID")
        
        # 4. Filter (Centroid)
        city_data = county_map[county_map.geometry.centroid.within(self.city_boundary.geometry.iloc[0])]
        
        # 5. Back to Lat/Lon for Mapping
        return city_data.to_crs(epsg=4326)

    def _save_map(self, gdf, col, cmap, title, filename, legend_label):
        """Save consistent maps"""
        fig, ax = plt.subplots(figsize=(10, 8))
        gdf.plot(column=col, cmap=cmap, linewidth=0.5, edgecolor='black', legend=True, 
                 legend_kwds={'label': legend_label}, ax=ax)
        ax.set_title(title, fontsize=16, fontweight='bold')
        ax.axis('off')
        fig.savefig(os.path.join(self.output_path, f"{filename}.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)
        print(f"   Saved: {filename}.png")

# MODULE 1: DEMOGRAPHICS
    
    def run_demographics(self):
        print("\n Running Demographics Module...")
        vars = {
            'B01003_001E': 'total_pop', 'B01002_001E': 'median_age',
            'B03002_003E': 'white_nh', 'B03002_004E': 'black_nh',
            'B03002_006E': 'asian_nh', 'B03002_012E': 'hispanic',
            'B01001_002E': 'male_total', 'B01001_026E': 'female_total',
            'B01001_003E': 'm_u5', 'B01001_004E': 'm_5_9', 'B01001_005E': 'm_10_14', 'B01001_006E': 'm_15_17',
            'B01001_007E': 'm_18_19','B01001_008E': 'm_20', 'B01001_009E': 'm_21', 'B01001_010E': 'm_22_24',
            'B01001_011E': 'm_25_29','B01001_012E': 'm_30_34', 'B01001_013E': 'm_35_39','B01001_014E': 'm_40_44',
            'B01001_015E': 'm_45_49','B01001_016E': 'm_50_54', 'B01001_017E': 'm_55_59','B01001_018E': 'm_60_61', 
            'B01001_019E': 'm_62_64', 'B01001_027E': 'f_u5', 'B01001_028E': 'f_5_9', 'B01001_029E': 'f_10_14', 
            'B01001_030E': 'f_15_17', 'B01001_031E': 'f_18_19','B01001_032E': 'f_20', 'B01001_033E': 'f_21', 
            'B01001_034E': 'f_22_24', 'B01001_035E': 'f_25_29','B01001_036E': 'f_30_34', 'B01001_037E': 'f_35_39',
            'B01001_038E': 'f_40_44', 'B01001_039E': 'f_45_49','B01001_040E': 'f_50_54', 'B01001_041E': 'f_55_59',
            'B01001_042E': 'f_60_61', 'B01001_043E': 'f_62_64', 'B09020_001E': 'pop_65_plus'
        }
        
        df = self._get_data_for_city(vars)
        
        # Calc
        df['area_sq_miles'] = df['ALAND'] / 2589988
        df['pop_density'] = df['total_pop'] / df['area_sq_miles']
        df['pct_non_white'] = ((df['total_pop'] - df['white_nh']) / df['total_pop']) * 100
        df['other_nh'] = df['total_pop'] - (df['white_nh'] + df['black_nh'] + df['asian_nh'] + df['hispanic'])
        
        # Age Buckets
        df['Under 18'] = df[['m_u5','m_5_9','m_10_14','m_15_17', 'f_u5','f_5_9','f_10_14','f_15_17']].sum(axis=1)
        df['18-24'] = df[['m_18_19','m_20','m_21','m_22_24', 'f_18_19','f_20','f_21','f_22_24']].sum(axis=1)
        df['25-34'] = df[['m_25_29','m_30_34', 'f_25_29','f_30_34']].sum(axis=1)
        df['35-44'] = df[['m_35_39','m_40_44', 'f_35_39','f_40_44']].sum(axis=1)
        df['45-54'] = df[['m_45_49','m_50_54', 'f_45_49','f_50_54']].sum(axis=1)
        df['55-64'] = df[['m_55_59','m_60_61','m_62_64', 'f_55_59','f_60_61','f_62_64']].sum(axis=1)
        df['65+']   = df['pop_65_plus']
        
        totals = df.sum(numeric_only=True)

        # Maps
        # 1. Density
        self._save_map(df, 'pop_density', 'Reds', 'Population Density (per SqMi)', '01_Map_Density', 'People/SqMi')
        
        # 2. Diversity 
        self._save_map(df, 'pct_non_white', 'Blues', 'Diversity (% Non-White)', '02_Map_Diversity', '% Non-White')
        
        # 3. Median Age
        valid_age = df[df['median_age'] > 0]
        self._save_map(valid_age, 'median_age', 'Purples_r', 'Median Age', '03_Map_MedianAge', 'Age (Years)')
        
        # Charts
        # Age
        fig, ax = plt.subplots(figsize=(10, 6))
        cols = ['Under 18', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']
        vals = totals[cols]
        bars = ax.bar(cols, vals, color='#5F9EA0', edgecolor='black')
        for b in bars: ax.text(b.get_x()+b.get_width()/2, b.get_height(), f"{int(b.get_height()):,}", ha='center', va='bottom', fontweight='bold')
        ax.set_title(f"{self.city_name} Age Composition", fontsize=16, fontweight='bold')
        fig.savefig(os.path.join(self.output_path, "04_Age_Composition.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

        # Race
        fig, ax = plt.subplots(figsize=(10, 6))
        r_cols = ['white_nh', 'asian_nh', 'black_nh', 'hispanic', 'other_nh']
        r_labs = ['White', 'Asian', 'Black', 'Hispanic', 'Others']
        r_vals = totals[r_cols]
        r_pcts = (r_vals / r_vals.sum()) * 100
        bars = ax.bar(r_labs, r_vals, color=['#d3d3d3', '#4682B4', '#FF7F50', '#2E8B57', '#DAA520'], edgecolor='black')
        for i, b in enumerate(bars): ax.text(b.get_x()+b.get_width()/2, b.get_height(), f"{int(b.get_height()):,}\n({r_pcts.iloc[i]:.1f}%)", ha='center', va='bottom', fontweight='bold')
        ax.set_title("Racial & Ethnic Composition", fontsize=16, fontweight='bold')
        fig.savefig(os.path.join(self.output_path, "05_Racial_Composition.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)
        
        # Sex
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.pie([totals['male_total'], totals['female_total']], labels=['Male', 'Female'], autopct='%1.1f%%', colors=['#89CFF0', '#F4C2C2'], startangle=90)
        ax.set_title("Sex Composition", fontsize=16, fontweight='bold')
        fig.savefig(os.path.join(self.output_path, "06_Sex_Composition.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

    # MODULE 2: HOUSING
    def run_housing(self):
        print("\n Running Housing Module...")
        vars = {
            'B25001_001E': 'total_housing_units', 'B25003_001E': 'occupied_units',
            'B25003_002E': 'owner_occupied', 'B25003_003E': 'renter_occupied', 'B25002_003E': 'vacant_units',
            'B25064_001E': 'median_gross_rent', 'B25077_001E': 'median_home_value',
            'B25070_001E': 'renter_universe_income', 'B25070_007E': 'rent_30_34_pct',
            'B25070_008E': 'rent_35_39_pct', 'B25070_009E': 'rent_40_49_pct', 'B25070_010E': 'rent_50_plus_pct'
        }
        df = self._get_data_for_city(vars)
        
        # Calc
        df['pct_renter'] = (df['renter_occupied'] / df['occupied_units']) * 100
        df['burdened_renters'] = df['rent_30_34_pct'] + df['rent_35_39_pct'] + df['rent_40_49_pct'] + df['rent_50_plus_pct']
        df['pct_cost_burdened'] = (df['burdened_renters'] / df['renter_universe_income']) * 100
        
        totals = df.sum(numeric_only=True)
        city_pct_renter = (totals['renter_occupied'] / totals['occupied_units']) * 100
        city_pct_burden = (totals['burdened_renters'] / totals['renter_universe_income']) * 100

        # Maps
        self._save_map(df, 'pct_renter', 'YlGnBu', '% Renter Occupied Housing', '07_Map_RenterShare', '% Renter')
        self._save_map(df, 'median_gross_rent', 'Greens', 'Median Gross Rent ($)', '08_Map_MedianRent', 'Rent ($)')
        self._save_map(df, 'pct_cost_burdened', 'RdYlGn_r', 'Renters Cost Burdened (>30% Income)', '09_Map_CostBurden', '% Burdened')
        
        # Charts
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.pie([city_pct_renter, 100-city_pct_renter], labels=['Renters', 'Owners'], autopct='%1.1f%%', colors=['#87CEFA', '#FFB6C1'], startangle=90)
        ax.set_title("Housing Tenure Split", fontsize=16, fontweight='bold')
        fig.savefig(os.path.join(self.output_path, "10_Chart_Tenure.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)
        
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.pie([city_pct_burden, 100-city_pct_burden], labels=['Burdened', 'Affordable'], autopct='%1.1f%%', colors=['#FF6347', '#90EE90'], startangle=140)
        ax.set_title("Renter Affordability Status", fontsize=16, fontweight='bold')
        fig.savefig(os.path.join(self.output_path, "11_Chart_CostBurden.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

    # MODULE 3: TRANSPORTATION
    def run_transportation(self):
        print("\n Running Transportation Module...")
        vars = {
            'B08301_001E': 'total_commuters', 'B08301_003E': 'drove_alone',
            'B08301_004E': 'carpooled', 'B08301_010E': 'public_transit',
            'B08301_018E': 'bicycle', 'B08301_019E': 'walked', 'B08301_021E': 'worked_from_home',
            'B08201_001E': 'total_households_veh', 'B08201_002E': 'no_vehicle',
            'B08201_003E': 'one_vehicle', 'B08201_004E': 'two_vehicles',
            'B08201_005E': 'three_vehicles', 'B08201_006E': 'four_plus_vehicles'
        }
        df = self._get_data_for_city(vars)

        # Calculations
        df['pct_transit'] = (df['public_transit'] / df['total_commuters']) * 100
        df['pct_active']  = ((df['bicycle'] + df['walked']) / df['total_commuters']) * 100
        df['pct_no_car'] = (df['no_vehicle'] / df['total_households_veh']) * 100
        
        totals = df.sum(numeric_only=True)

        # Maps
        self._save_map(df, 'pct_transit', 'Blues', 'Public Transit Usage (Commute)', '12_Map_TransitUsage', '% Transit')
        self._save_map(df, 'pct_active', 'Greens', 'Active Commuting (Walk/Bike)', '13_Map_ActiveTransport', '% Active')
        self._save_map(df, 'pct_no_car', 'OrRd', 'Zero-Car Households', '14_Map_ZeroCar', '% No Car')

        # Chart: Mode Split
        fig, ax = plt.subplots(figsize=(10, 6))
        modes = ['Drive/Carpool', 'Public Transit', 'Walk', 'Bike', 'Work from Home']
        counts = [
            totals['drove_alone'] + totals['carpooled'], totals['public_transit'],
            totals['walked'], totals['bicycle'], totals['worked_from_home']
        ]
        pcts = [(x / totals['total_commuters'])*100 for x in counts]
        bars = ax.bar(modes, counts, color=['#808080', '#4682B4', '#2E8B57', '#32CD32', '#DAA520'], edgecolor='black')
        for i, b in enumerate(bars): 
            ax.text(b.get_x()+b.get_width()/2, b.get_height(), f"{int(b.get_height()):,}\n({pcts[i]:.1f}%)", ha='center', va='bottom', fontweight='bold')
        ax.set_title("Commute Mode Split", fontsize=16, fontweight='bold')
        fig.savefig(os.path.join(self.output_path, "15_Chart_ModeSplit.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

        # Chart: Car Ownership
        fig, ax = plt.subplots(figsize=(6, 6))
        two_plus = totals['two_vehicles'] + totals['three_vehicles'] + totals['four_plus_vehicles']
        ax.pie([totals['no_vehicle'], totals['one_vehicle'], two_plus], labels=['0 Cars', '1 Car', '2+ Cars'], autopct='%1.1f%%', colors=['#2E8B57', '#87CEFA', '#D3D3D3'], startangle=90)
        ax.set_title("Household Vehicle Ownership", fontsize=16, fontweight='bold')
        centre_circle = plt.Circle((0,0),0.70,fc='white')
        fig.gca().add_artist(centre_circle)
        fig.savefig(os.path.join(self.output_path, "16_Chart_CarOwnership.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

    # MODULE 4: EMPLOYMENT
    def run_employment(self):
        print("\n Running Employment Module...")
        # Note: Files start at 17 now
        vars = {
            'B19013_001E': 'median_hh_income', 'B23025_001E': 'pop_16_plus',
            'B23025_002E': 'in_labor_force', 'B23025_004E': 'employed', 'B23025_005E': 'unemployed'
        }
        df = self._get_data_for_city(vars)
        df['unemployment_rate'] = (df['unemployed'] / df['in_labor_force']) * 100
        totals = df.sum(numeric_only=True)
        
        valid_income = df[df['median_hh_income'] > 0]
        self._save_map(valid_income, 'median_hh_income', 'Greens', 'Median Household Income ($)', '17_Map_Income', 'Income ($)')
        self._save_map(df, 'unemployment_rate', 'Reds', 'Unemployment Rate (%)', '18_Map_Unemployment', 'Rate (%)')
        
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.pie([totals['employed'], totals['unemployed']], labels=['Employed', 'Unemployed'], autopct='%1.1f%%', colors=['#90EE90', '#FF6347'], startangle=90)
        ax.set_title("Labor Force Status", fontsize=16, fontweight='bold')
        fig.savefig(os.path.join(self.output_path, "19_Chart_LaborForce.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

    # MODULE 5: EDUCATION
    def run_education(self):
        print("\n Running Education Module...")
        vars = {
            'B15003_001E': 'pop_25_plus', 'B15003_017E': 'hs_diploma', 'B15003_018E': 'ged',
            'B15003_019E': 'some_college_1', 'B15003_020E': 'some_college_plus', 'B15003_021E': 'associate_degree',
            'B15003_022E': 'bachelors_degree', 'B15003_023E': 'masters_degree', 
            'B15003_024E': 'professional_degree', 'B15003_025E': 'doctorate_degree',
            'B14001_001E': 'total_enrolled_pop', 'B14001_008E': 'enrolled_undergrad', 'B14001_009E': 'enrolled_grad_prof'
        }
        df = self._get_data_for_city(vars)
        
        df['higher_ed_pop'] = (df['bachelors_degree'] + df['masters_degree'] + df['professional_degree'] + df['doctorate_degree'])
        df['pct_bachelors_plus'] = (df['higher_ed_pop'] / df['pop_25_plus']) * 100
        df['advanced_degree_pop'] = (df['masters_degree'] + df['professional_degree'] + df['doctorate_degree'])
        df['pct_advanced_degree'] = (df['advanced_degree_pop'] / df['pop_25_plus']) * 100
        df['total_college_students'] = df['enrolled_undergrad'] + df['enrolled_grad_prof']
        
        totals = df.sum(numeric_only=True)
        
        self._save_map(df, 'pct_bachelors_plus', 'Blues', "Bachelor's Degree or Higher (%)", '20_Map_BachelorsPlus', '%')
        self._save_map(df, 'pct_advanced_degree', 'Purples', "Graduate/Prof Degree (%)", '21_Map_AdvancedDegrees', '%')
        self._save_map(df, 'total_college_students', 'Oranges', "Student Population (Count)", '22_Map_StudentPop', 'Students')
        
        # Chart
        fig, ax = plt.subplots(figsize=(10, 6))
        cols = [totals['some_college_1']+totals['some_college_plus']+totals['associate_degree'], totals['bachelors_degree'], totals['masters_degree'], totals['professional_degree']+totals['doctorate_degree']]
        hs_less = totals['pop_25_plus'] - sum(cols)
        counts = [hs_less] + cols
        labels = ['HS or Less', 'Some College', "Bachelor's", "Master's", 'Prof/PhD']
        pcts = [(x / totals['pop_25_plus'])*100 for x in counts]
        
        bars = ax.bar(labels, counts, color='#4682B4', edgecolor='black')
        for i, b in enumerate(bars): ax.text(b.get_x()+b.get_width()/2, b.get_height(), f"{int(b.get_height()):,}\n({pcts[i]:.1f}%)", ha='center', va='bottom', fontweight='bold')
        ax.set_title("Educational Attainment", fontsize=16, fontweight='bold')
        fig.savefig(os.path.join(self.output_path, "23_Chart_Education.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

    def generate_report(self):
        print("\n Compiling PowerPoint Report...")
        prs = Presentation()
        prs.slide_width = Inches(13.333)
        prs.slide_height = Inches(7.5)
        
        files = [f for f in os.listdir(self.output_path) if f.endswith(".png") and f[0].isdigit()]
        files.sort()
        
        for filename in files:
            slide = prs.slides.add_slide(prs.slide_layouts[6])
            title_text = filename.replace(".png", "").split("_", 1)[1].replace("_", " ")
            
            # Title
            title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.4), Inches(5), Inches(1))
            p = title_box.text_frame.paragraphs[0]
            p.text = title_text
            p.font.size = Pt(28); p.font.bold = True; p.font.name = 'Arial'
            
            # Image
            img_path = os.path.join(self.output_path, filename)
            slide.shapes.add_picture(img_path, Inches(5.0), Inches(0.5), height=Inches(6.5))
            
        output_pptx = os.path.join(self.output_path, f"{self.city_name}_Report.pptx")
        prs.save(output_pptx)
        print(f"Report Saved: {output_pptx}")

    def run_all(self):
        print(f"STARTING FULL REPORT FOR {self.city_name}...")
        self.run_demographics()
        self.run_housing()
        self.run_transportation()
        self.run_employment()
        self.run_education()
        self.generate_report()
        print("\n COMPLETE.")

In [ ]:
# 1. DEFINE YOUR KEY
MY_KEY = "YOUR_API_KEY"

# 2. RUN THE AGENT
agent = MappingAgent(
    city_name="CITY_NAME", 
    state_abbrev="STATE", 
    county_name="COUNTY", 
    api_key=MY_KEY, 
    output_root="YOUR_OUTPUT_PATH"
)

agent.run_all()